In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Install the libraries

In [1]:
!pip install transformers torch fastapi uvicorn[standard] ngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 73.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.8/510.8 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 102.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.2/452.2 kB 32.0 MB/s eta 0:00:00


In [3]:
model = "nlptown/bert-base-multilingual-uncased-sentiment"

In [4]:
tokens = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [5]:
tokens

BertTokenizerFast(name_or_path='nlptown/bert-base-multilingual-uncased-sentiment', vocab_size=105879, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [6]:
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

Method to do semantic analysis

In [7]:
def sentiment_score(review):
    tokens_text = tokens(review, return_tensors='pt', truncation=True, padding=True)
    out = model(**tokens_text)
    scores = torch.nn.functional.softmax(out.logits, dim=1)
    predicted_label = torch.argmax(scores, dim=1).item()
    return predicted_label

In [8]:
sentiment_score("I love this score as it is great")

4

In [9]:
import fastapi
from pydantic import BaseModel
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [16]:
app = fastapi.FastAPI()

In [12]:
class InputText(BaseModel):
    text: str

In [17]:
@app.post("/predict")
async def predict_sentiment(review: InputText):
    tokens_text = tokens(review, return_tensors='pt', truncation=True, padding=True)
    out = model(**tokens_text)
    scores = torch.nn.functional.softmax(out.logits, dim=1)
    predicted_label = torch.argmax(scores, dim=1).item()
    return {"sentiment_label": int(predicted_label)}

In [15]:
!pip install pyngrok
from pyngrok import ngrok

In [19]:
# Run FastAPI app in background
!uvicorn app:app --host 0.0.0.0 --port 8000 &

ERROR:    Error loading ASGI app. Could not import module "app".


In [1]:
# Expose public URL
public_url = ngrok.connect(8000)
print(public_url)

NameError: name 'ngrok' is not defined

In [2]:
import requests
resp = requests.post(f"{public_url}/predict", json={"text": "I love this movie!"})
resp.json()

NameError: name 'public_url' is not defined

In [6]:
cd /content/drive/MyDrive/Github/SentimentAnalysis_Bert

/content/drive/MyDrive/Github/SentimentAnalysis_Bert


In [7]:
%%writefile Dockerfile
FROM python:3.9-slim

WORKDIR /app
COPY . /app

RUN pip install --no-cache-dir fastapi uvicorn[standard] transformers torch

EXPOSE 8000
CMD ["uvicorn", "app:app", "--host", "0.0.0.0", "--port", "8000"]

Writing Dockerfile


In [8]:
!pwd

/content/drive/MyDrive/Github/SentimentAnalysis_Bert


In [9]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	Analysis.ipynb
	Dockerfile
	app.py

nothing added to commit but untracked files present (use "git add" to track)


In [10]:
#!git init
#!git remote add origin https://github.com/<yourusername>/<yourrepo>.git
!git add .
!git commit -m "BERT sentiment API"
!git push origin master

Author identity unknown

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@a33910ca59b6.(none)')
error: src refspec master does not match any
error: failed to push some refs to 'https://github.com/SalilVishnuKapur/SentimentAnalysis_Bert.git'


In [12]:
!git config --global user.email "salilvishnukapur@gmail.com"

In [13]:
!git config --global user.name "Salil Vishnu Kapur"